In [194]:
import pandas as pd
import carbonpipeline.cli as cli
from carbonpipeline.constants import *
import cfgrib
import xarray as xr

In [195]:
df = pd.read_csv("predictors.csv")

In [196]:
filtered_df = cli.filtered_and_renamed_columns(df, COLUMN_NAME_MAPPING)
filtered_df['timestamp'] = pd.to_datetime(filtered_df['timestamp'])

In [197]:
filtered_df

,CO2,G,H,LE,LW_IN,LW_OUT,NETRAD,PA,PPFD_IN,PPFD_OUT,P,RH,SW_IN,SW_OUT,TA,USTAR,VPD,WD,WS,timestamp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 00:00:00
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 01:00:00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 02:00:00
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 03:00:00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236683,-0.301087,-0.167411,-0.173071,-0.162175,-0.360071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,-0.016972,NaN,-0.495032,NaN,NaN,2020-12-31 19:00:00
236684,-0.301083,-0.167410,-0.173181,-0.162261,-0.360130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,-0.016956,NaN,-0.495005,NaN,NaN,2020-12-31 20:00:00
236685,-0.301080,-0.167409,-0.173203,-0.162386,-0.360191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,-0.016956,NaN,-0.494991,NaN,NaN,2020-12-31 21:00:00
236686,-0.301077,-0.167408,-0.173198,-0.162516,-0.360256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,-0.016956,NaN,-0.494991,NaN,NaN,2020-12-31 22:00:00


In [198]:
input_vars = ['CO2', 'G', 'LW_IN']
renamed_vars = map(lambda var: "CS, " + var, input_vars)
a = filtered_df.rename(columns=dict(zip(input_vars, renamed_vars)))
a

,"CS, CO2","CS, G",H,LE,"CS, LW_IN",LW_OUT,NETRAD,PA,PPFD_IN,PPFD_OUT,P,RH,SW_IN,SW_OUT,TA,USTAR,VPD,WD,WS,timestamp
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 00:00:00
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 01:00:00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 02:00:00
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 03:00:00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236683,-0.301087,-0.167411,-0.173071,-0.162175,-0.360071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,-0.016972,NaN,-0.495032,NaN,NaN,2020-12-31 19:00:00
236684,-0.301083,-0.167410,-0.173181,-0.162261,-0.360130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,-0.016956,NaN,-0.495005,NaN,NaN,2020-12-31 20:00:00
236685,-0.301080,-0.167409,-0.173203,-0.162386,-0.360191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,-0.016956,NaN,-0.494991,NaN,NaN,2020-12-31 21:00:00
236686,-0.301077,-0.167408,-0.173198,-0.162516,-0.360256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5,NaN,-0.016956,NaN,-0.494991,NaN,NaN,2020-12-31 22:00:00


In [199]:
import numpy as np

df = a.copy()
df[[("ERA5, " + var) for var in input_vars]] = np.nan
df

,"CS, CO2","CS, G",H,LE,"CS, LW_IN",LW_OUT,NETRAD,PA,PPFD_IN,PPFD_OUT,...,SW_OUT,TA,USTAR,VPD,WD,WS,timestamp,"ERA5, CO2","ERA5, G","ERA5, LW_IN"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 00:00:00,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 01:00:00,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 02:00:00,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 03:00:00,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1994-01-01 04:00:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236683,-0.301087,-0.167411,-0.173071,-0.162175,-0.360071,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.016972,NaN,-0.495032,NaN,NaN,2020-12-31 19:00:00,NaN,NaN,NaN
236684,-0.301083,-0.167410,-0.173181,-0.162261,-0.360130,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.016956,NaN,-0.495005,NaN,NaN,2020-12-31 20:00:00,NaN,NaN,NaN
236685,-0.301080,-0.167409,-0.173203,-0.162386,-0.360191,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.016956,NaN,-0.494991,NaN,NaN,2020-12-31 21:00:00,NaN,NaN,NaN
236686,-0.301077,-0.167408,-0.173198,-0.162516,-0.360256,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.016956,NaN,-0.494991,NaN,NaN,2020-12-31 22:00:00,NaN,NaN,NaN


In [200]:
raw_cols = df.columns.tolist()

levels = []
for col in raw_cols:
    parts = col.split(', ')        
    if len(parts) == 2:
        src, var = parts           # Inputs vars => has both CS and ERA5 levels
    else:
        src, var = 'CS', parts[0]  # Vars not needed to be requested => doesn't have ERA5 level
    levels.append((var, src))

df.columns = pd.MultiIndex.from_tuples(levels, names=['variable','source']) # Variable for outer level, source en inner level

df = df.sort_index(axis=1, level='variable') # Group by variable
df

variable       CO2              G              H        LE     LW_IN       \
source          CS ERA5        CS ERA5        CS        CS        CS ERA5   
0              NaN  NaN       NaN  NaN       NaN       NaN       NaN  NaN   
1              NaN  NaN       NaN  NaN       NaN       NaN       NaN  NaN   
2              NaN  NaN       NaN  NaN       NaN       NaN       NaN  NaN   
3              NaN  NaN       NaN  NaN       NaN       NaN       NaN  NaN   
4              NaN  NaN       NaN  NaN       NaN       NaN       NaN  NaN   
...            ...  ...       ...  ...       ...       ...       ...  ...   
236683   -0.301087  NaN -0.167411  NaN -0.173071 -0.162175 -0.360071  NaN   
236684   -0.301083  NaN -0.167410  NaN -0.173181 -0.162261 -0.360130  NaN   
236685   -0.301080  NaN -0.167409  NaN -0.173203 -0.162386 -0.360191  NaN   
236686   -0.301077  NaN -0.167408  NaN -0.173198 -0.162516 -0.360256  NaN   
236687   -0.301075  NaN -0.167407  NaN -0.173225 -0.162551 -0.360322  NaN   

variable LW_OUT NETRAD  ... PPFD_OUT  RH SW_IN SW_OUT        TA USTAR  \
source       CS     CS  ...       CS  CS    CS     CS        CS    CS   
0           NaN    NaN  ...      NaN NaN   NaN    NaN       NaN   NaN   
1           NaN    NaN  ...      NaN NaN   NaN    NaN       NaN   NaN   
2           NaN    NaN  ...      NaN NaN   NaN    NaN       NaN   NaN   
3           NaN    NaN  ...      NaN NaN   NaN    NaN       NaN   NaN   
4           NaN    NaN  ...      NaN NaN   NaN    NaN       NaN   NaN   
...         ...    ...  ...      ...  ..   ...    ...       ...   ...   
236683      NaN    NaN  ...      NaN NaN  -0.5    NaN -0.016972   NaN   
236684      NaN    NaN  ...      NaN NaN  -0.5    NaN -0.016956   NaN   
236685      NaN    NaN  ...      NaN NaN  -0.5    NaN -0.016956   NaN   
236686      NaN    NaN  ...      NaN NaN  -0.5    NaN -0.016956   NaN   
236687      NaN    NaN  ...      NaN NaN  -0.5    NaN -0.016956   NaN   

variable       VPD  WD  WS           timestamp  
source          CS  CS  CS                  CS  
0              NaN NaN NaN 1994-01-01 00:00:00  
1              NaN NaN NaN 1994-01-01 01:00:00  
2              NaN NaN NaN 1994-01-01 02:00:00  
3              NaN NaN NaN 1994-01-01 03:00:00  
4              NaN NaN NaN 1994-01-01 04:00:00  
...            ...  ..  ..                 ...  
236683   -0.495032 NaN NaN 2020-12-31 19:00:00  
236684   -0.495005 NaN NaN 2020-12-31 20:00:00  
236685   -0.494991 NaN NaN 2020-12-31 21:00:00  
236686   -0.494991 NaN NaN 2020-12-31 22:00:00  
236687   -0.494968 NaN NaN 2020-12-31 23:00:00  

[236688 rows x 23 columns]

In [201]:
miss = filtered_df[filtered_df[filtered_df.columns.drop('timestamp')].isnull().any(axis=1)].copy()
miss['year']  = miss['timestamp'].dt.year
miss['month'] = miss['timestamp'].dt.month
miss['day'] = miss['timestamp'].dt.day

In [202]:
combined = cli.merge_datasets()
combined

failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed to set key 'missingValue' to np.float32(3.4028235e+38)
failed t

number   step  depthBelowLandLayer  \
time                latitude longitude                                       
1994-01-01 00:00:00 75.125   44.875          0 0 days                  0.0   
                             45.125          0 0 days                  0.0   
                    74.875   44.875          0 0 days                  0.0   
                             45.125          0 0 days                  0.0   
1994-01-01 01:00:00 75.125   44.875          0 0 days                  0.0   
...                                        ...    ...                  ...   
1994-01-01 22:00:00 74.875   45.125          0 0 days                  0.0   
1994-01-01 23:00:00 75.125   44.875          0 0 days                  0.0   
                             45.125          0 0 days                  0.0   
                    74.875   44.875          0 0 days                  0.0   
                             45.125          0 0 days                  0.0   

                                                valid_time  \
time                latitude longitude                       
1994-01-01 00:00:00 75.125   44.875    1994-01-01 00:00:00   
                             45.125    1994-01-01 00:00:00   
                    74.875   44.875    1994-01-01 00:00:00   
                             45.125    1994-01-01 00:00:00   
1994-01-01 01:00:00 75.125   44.875    1994-01-01 01:00:00   
...                                                    ...   
1994-01-01 22:00:00 74.875   45.125    1994-01-01 22:00:00   
1994-01-01 23:00:00 75.125   44.875    1994-01-01 23:00:00   
                             45.125    1994-01-01 23:00:00   
                    74.875   44.875    1994-01-01 23:00:00   
                             45.125    1994-01-01 23:00:00   

                                        volumetric_soil_water_layer_1  \
time                latitude longitude                                  
1994-01-01 00:00:00 75.125   44.875                     -9.499490e-07   
                             45.125                     -9.499490e-07   
                    74.875   44.875                     -9.499490e-07   
                             45.125                     -9.499490e-07   
1994-01-01 01:00:00 75.125   44.875                      9.577721e-06   
...                                                               ...   
1994-01-01 22:00:00 74.875   45.125                     -7.696450e-06   
1994-01-01 23:00:00 75.125   44.875                      5.960464e-08   
                             45.125                      5.960464e-08   
                    74.875   44.875                      5.960464e-08   
                             45.125                      5.960464e-08   

                                        soil_temperature_level_1  \
time                latitude longitude                             
1994-01-01 00:00:00 75.125   44.875                   272.939819   
                             45.125                   272.997925   
                    74.875   44.875                   273.009216   
                             45.125                   273.068268   
1994-01-01 01:00:00 75.125   44.875                   272.939789   
...                                                          ...   
1994-01-01 22:00:00 74.875   45.125                   273.044586   
1994-01-01 23:00:00 75.125   44.875                   272.963165   
                             45.125                   272.980591   
                    74.875   44.875                   273.026398   
                             45.125                   273.044128   

                                        volumetric_soil_water_layer_2  \
time                latitude longitude                                  
1994-01-01 00:00:00 75.125   44.875                          0.000004   
                             45.125                          0.000004   
                    74.875   44.875                          0.000004   
                  

In [203]:
combined = combined.rename(columns=SHORTNAME_TO_FULLNAME)

In [204]:
combined.index.names

FrozenList(['time', 'latitude', 'longitude'])